In [4]:
import import_ipynb
from library import *

importing Jupyter notebook from library.ipynb
importing Jupyter notebook from job.ipynb
importing Jupyter notebook from jobqueue.ipynb
importing Jupyter notebook from network.ipynb
importing Jupyter notebook from dataplane.ipynb


In [3]:
class system_manager():
    def __init__(self):
        self.network = network()
        self.job_waiting_queue = job_waiting_queue()
        self.backlog = backlog()
        self.activated_job = [None]*10 # 현재 system 안에서 돌아가고 있는 jobs.
        self.activated_job_num = 0
        self.src_dst_pair = []
        self.completed = []
        for src in range(5):
            for dst in range(5):
                if src == dst:
                    continue
                self.src_dst_pair.append([src, dst])
    
    
    
    def create_job(self, time):
        job_num = np.random.poisson(lam=1, size=len(self.src_dst_pair))
        for index in range(len(self.src_dst_pair)):
            src, dst = self.src_dst_pair[index]
            for _ in range(job_num[index]):
                model = random.randint(1, 3)
                new_job = job(src, dst, model, time)
                if len(self.job_waiting_queue) < 10:
                    self.job_waiting_queue.push(new_job)
                else:
                    self.backlog.push(new_job)
    
    
    
    def move_job(self):
        while len(self.job_waiting_queue) < 10:
            if len(self.backlog) == 0:
                break
            else:
                job = self.backlog.pop()
                self.job_waiting_queue.push(job)
                
                
                
    def assign_index(self):
        for idx, room in enumerate(self.activated_job):
            if room == None:
                self.job_waiting_queue[0].index = idx
                return idx
            
            
    
    def schedule(self, index, offloading_vector):
        job = self.job_waiting_queue.pop()
        self.activated_job[index] = job
        src, dst = job.source, job.destination

        for order, node in enumerate(offloading_vector):
            job.offloading.append(node)
            job.subtasks[order].proc_node = node
        
        '''
        print(f'src-dst : {job.source}, {job.destination}')
        print("offloading)")
        for k in job.offloading:
            print(k, end=' ')
        print()
        '''
        
        # transfer output to destination from last node
        if dst != job.offloading[-1]:
            job.offloading.append(dst)

        # transfer input to first node from source
        if src == job.offloading[0]:
            self.network.nodes[src].push(job)
        else:
            job.subtasks.insert(0, subtask(0, 3, -1, -1))
            path = self.network.routing(src, job.offloading[0])
            job.routing = path
            self.network.links[f'{path[0]}{path[1]}'].push(job)
            self.network.links[f'{path[0]}{path[1]}'].waiting += 3
            del job.routing[0]

        self.activated_job_num += 1
        
        return
                
                
                
    def schedule_job(self):
        while self.activated_job_num < 10 and len(self.job_waiting_queue) != 0:
            # 지금 job의 index 계산
            index = self.assign_index()
            job = self.job_waiting_queue.pop()
            self.activated_job[index] = job
            
            src, dst = job.source, job.destination
            
            # offloading scheduling
            for order in range(job.model+2):
                node = random.randint(0, 4)
                job.offloading.append(node)
                job.subtasks[order].proc_node = node
            
            print(f'src-dst : {job.source}, {job.destination}')
            print("offloading)")
            for k in job.offloading:
                print(k, end=' ')
            print()
            
            # transfer output data logic
            if dst != job.offloading[-1]:
                job.offloading.append(dst)
            
            # transfer input data logic
            if src == job.offloading[0]:
                self.network.nodes[src].push(job)
            else:
                job.subtasks.insert(0, subtask(0, 3, -1, -1))
                path = self.network.routing(src, job.offloading[0])
                job.routing = path
                self.network.links[f'{path[0]}{path[1]}'].push(job)
                self.network.links[f'{path[0]}{path[1]}'].waiting += 3
                del job.routing[0]
                
            self.activated_job_num += 1
            
        return
            
                
                
    
    def get_state(self):
        node_waiting_vector = [[0]*(5*10) for _ in range(5)]
        node_processing_vector = [[0]*(5*10) for _ in range(5)]
        
        # get node waiting vector
        for job_idx, job in enumerate(self.activated_job):
            if job == None:
                continue
            for subtask in job.subtasks:
                order, proc_node, rem_comp_demand = subtask.order, subtask.proc_node, subtask.comp_demand
                if order >= 0:
                    node_waiting_vector[proc_node][5*job_idx+order] = rem_comp_demand/100
            
        # get node processing vector
        for node_idx, node in enumerate(self.network.nodes):
            job_idx, order = node.cur_proc_job, node.cur_proc_subtask
            if job_idx != -1 and order != -1:
                node_processing_vector[node_idx][5*job_idx+order] = 1
                

        
        # node feature
        node_waiting_vector = np.array(node_waiting_vector)
        node_processing_vector = np.array(node_processing_vector)
        node_feature = np.concatenate((node_waiting_vector, node_processing_vector), axis=1)
        
        # link feature
        link_feature = []
        for key, link in self.network.links.items():
            link_feature.append([link.get_waiting()])
        link_feature = np.array(link_feature)
        
        return node_feature, link_feature
            
    
    
    def get_job_waiting_vector(self):
        job_waiting_vector = [[0]*(5+5) for _ in range(10)]
        
        for job_idx in range(len(self.job_waiting_queue)):
            job = self.job_waiting_queue[job_idx]
            src, dst = job.source, job.destination
            job_waiting_vector[job_idx][src] = -1
            job_waiting_vector[job_idx][dst] = 1
            
            for subtask in job.subtasks:
                order, comp_demand = subtask.order, subtask.comp_demand
                job_waiting_vector[job_idx][5+order] = comp_demand/100
        
        job_waiting_vector = np.array(job_waiting_vector)
        job_waiting_vector = job_waiting_vector.flatten()
        return job_waiting_vector
        
    
    def get_reward(self):
        average_speed = 0
        num = 0
        for job_idx, job in enumerate(self.activated_job):
            if job == None:
                continue
            num += 1
            average_speed += job.comp_delta/job.total_comp
            job.comp_delta = 0

        for job in self.completed:
            num += 1
            average_speed += job.comp_delta/job.total_comp
        
        if num==0:
            return 0

        self.completed = []

        return 10*average_speed/num
    

    def init_job_progress(self):
        for job in self.activated_job:
            if job == None:
                continue
            job.progress = 0
        return
        
        
    
    def step(self, time):
        completed_job_index = self.network.step(time)
        for index in completed_job_index:
            self.completed.append(self.activated_job[index])
            self.activated_job[index] = None
            self.activated_job_num -= 1